# Assignment 2 - Simple World

Welcome to Assignment 2 of the "Robot Programming with Lisp" course.

* Due Date: 02.11.2022, 23:59 CEST in your Github repository
* 7 points achievable

In this assignment you are a robot living in a 2D world. 
The world consist of ground, where you can stand, and walls, where you can not.
From your current location you can fly/teleport/phase/apperate anywhere instantly, but you can not land on walls. 
There are little treasures spread around the world.
Your goal is to collect all the treasures by teleporting the robot onto the treasures.

## How to

Execute the code segments one after another. Each chapter in this notebook serves its purpose.

* **Load Package** - How to load the package of the assignment
* **Prepared code** - Some prepared code, containing parameters and functions for the world. Don't touch that.
* **Assignment code** - Your assignment. This is where you need to write code. Solve the missing TODOs by implementing the functionality described.
* **Testing** - With these functions you can test your program. Add more code blocks here if you want to test specific functions.

You have completed the assignment when the robot picks up all the treasures in the world. Your grade depends on your individual implementation in the **Assignment code**.

## Load Package

Execute the code of the following section to load this assignments code.

In [ ]:
;; Adds the directory of this assignment into the ASDF registry
(let* ((lecture-path '(:absolute "home" "lectures" "robot_programming_with_lisp"))
       (assignment-path (append lecture-path '("02_simple_world")))
       (bullet-path (append lecture-path '("bullet_wrapper"))))
      (pushnew (make-pathname :directory assignment-path) asdf:*central-registry*)
      (pushnew (make-pathname :directory bullet-path) asdf:*central-registry*))

In [ ]:
;; Loads the ASDF system of this assignment
(asdf:load-system :simple-world)

In [ ]:
;; Makes this assignment as the current namespace
(in-package :simple-world)

## Prepared Code

This code is already done and doesn't need adjustments. Find your assignment in the next section, down below.

In [ ]:
;;; ## About the world
;;;
;;; In this assignment you have a robot living in a flat 2-dimensional world.
;;; This world only consist of symbols:
;;; robot (R), ground (G), walls (W) and treasures (A, B, C, D, E, F).
;;; Initially, the world only has ground and walls.
;;; It's state is kept in the variable *world-map*
;;; That *world-map* is a 2D array.
;;; The simulation reads and visualizes the array's content.

(defvar *world-map*
  #2A((w w w w w w w w w w w w w w w w)
      (w w g g g g g w g g g w g g g w)
      (w w g g g g g w g g g w g g g w)
      (w w w w g w w w w g w w w w g w)
      (w g g g g g g g g g g g g g g w)
      (w w w w g w w w w w w g w w w w)
      (w g g w g w g g g w g g g w w w)
      (w g g w g w g g g w g g g w w w)
      (w g g g g g g g g w g g g w w w)
      (w g g w g w g g g w w w w w w w)
      (w w w w g w w w w w w w w w w w)
      (w g g w g w g g g w w w w w w w)
      (w g g g g w g g g w w w w w w w)
      (w g g w g g g g g w w w w w w w)
      (w w w w w w w w w w w w w w w w))
  "2D grid world.
The following symbols are used to represent things in the world:
- W for wall cell
- G for ground
- R for robot
- T for treasure.")

(defun initialize-walls ()
  "Cleans the *world-map* array and initializes the walls and the ground."
  (setf *world-map* (make-array '(15 16) :initial-element 'g))
  (loop for i from 0 to 14
        do (setf (aref *world-map* i 0) 'w)
           (setf (aref *world-map* i 15) 'w)
           (setf (aref *world-map* 0 i) 'w)
           (setf (aref *world-map* 14 i) 'w))
  (mapcar (lambda (x-y)
            (setf (aref *world-map* (first x-y) (second x-y)) 'w))
          '((0 15)
            (1 1) (1 7) (1 11)
            (2 1) (2 7) (2 11) (3 1) (3 2) (3 3) (3 5) (3 6) (3 7) (3 8) (3 10) (3 11) (3 12) (3 13)
            (5 1) (5 2) (5 3) (5 5) (5 6) (5 7) (5 8) (5 9) (5 10) (5 12) (5 13) (5 14)
            (6 3) (6 5) (6 9) (6 13) (6 14)
            (7 3) (7 5) (7 9) (7 13) (7 14)
            (8 9) (8 13) (8 14)
            (9 3) (9 5) (9 9) (9 10) (9 11) (9 12) (9 13) (9 14)
            (10 1) (10 2) (10 3) (10 5) (10 6) (10 7) (10 8) (10 9) (10 10) (10 11) (10 12) (10 13) (10 14)
            (11 3) (11 5) (11 9) (11 10) (11 11) (11 12) (11 13) (11 14)
            (12 5) (12 9) (12 10) (12 11) (12 12) (12 13) (12 14)
            (13 3) (13 9) (13 10) (13 11) (13 12) (13 13) (13 14)
            (14 15))))

(defconstant +battery-capacity+ 50
  "Power of the robots battery in digits.")

(defconstant +treasure-num+ 4
  "The number of treasures that exist inthe world.")

(defparameter *treasures* '()
  "The list of treasures in the world.
Call the function `reset-treasures-list' to reset the `*treasures*' list.
It will then contain 4 randomly chosen symbols from the list '(a b c d e f).")

(defun reset-treasures-list ()
  (let ((symbols '(a b c d e f)))
    (setf symbols (remove (nth (random (length symbols)) symbols) symbols))
    (setf symbols (remove (nth (random (length symbols)) symbols) symbols))
    (setf *treasures* (alexandria:shuffle symbols))))

(defvar *treasures-found* 0
  "Determines the current amount of treasures found by the robot.
Increase this whenever a treasure is collected.")

(defvar *battery-left* +battery-capacity+
  "Describes how much power is left in the robots battery.
Decrease this value by 1 for every move made.")

(defvar *robot-coords* '(nil nil)
  "List of length 2, containing the x and y coordinate of the robot.")

## Assignment code

Here is the code of this assignment. Fill in the gaps where it says TODO.

Your assignment consists of two major tasks:
* initializing the world (`initialize-world`)
* moving the robot around to collect treasures (`discover-world`)

In [ ]:
(defun place-object (x y symbol)
  "If the given coordinate (x y) contains ground, replace it with the `symbol'."
  ;; TODO Implement
  )

In [ ]:
(defun place-robot (x y)
  "Uses `place-object' to set the robot symbol 'r in the `*world-map*'
and updates the robot's coordinates `*robot-coords*'.
The robot can only be placed on the ground."
  ;; TODO Implement
  )

In [ ]:
(defun initialize-world ()
  "This function initializes the 2D array map.
Write (initialize-world) and (visualize-simulation) to see the current world status.
Or just write *world-map* to see the 2D array.
1. It puts the symbols in the list `*treasures*' at random positions on the map.
If the treasure falls on a wall, the robot would not be able to collect it, so a wall stays a wall,
and you can only put treasures on the ground. After calling this function, there should be 4 symbols
in the world, a subset of the list '(a b c d e f).
2. It sets the robot on a random place on the ground in the `*world-map*'.
3. It resets all global variables so that one could play the treasure hunting game multiple
times in a row."
  (initialize-walls)
  (btr-wrapper::init-world)
  ;; TODO Implement
  )

In [ ]:
(defun symbol-at-pos (x y symbol)
  "Checks, if the symbol at position x and y is equal to the given symbol `symbol'."
  ;; TODO Implement
  )

In [ ]:
(defun find-object-coordinates (symbol)
  "Searches the 2D `*world-map*' for the given `symbol' and returns the coordinates."
  ;; TODO Implement
  )

In [ ]:
(defun move-robot (x y)
  "Flies the robot to the grid cell (x, y) in the world (indexing starts with 0).
If the cell is a wall or a treasure, anything except ground, the position is blocked
and nothing happens. Also, if `x' or `y' are outside the map boundaries, nothing should happen.
If the robot moves, it loses one battery charge. If the robot stands below a treasure,
you should incease the `*treasures-found*' counter.
This function will alter the world state (value of *world-map*).
Finally the simulation is updated."
  ;; TODO Implement

  ;; Visualize
  (visualize-simulation))

In [ ]:
(defun discover-world ()
  "Autonomously flies the robot in the world until all the treasure is collected.
The robot can only fly `+battery-capacity+' times.
The robot has access to the world state `*world-map*' and can read the coordinates of treasures.
Use `move-robot' to travel to each treasure, which collects the treasure above the robot."
  ;; TODO Implement
  )

## Testing

To test your whole implementation: *Restart the kernel and run all cells...*, which is the `>>` button at the top. 

For intermediate testing, reset the world with

In [ ]:
(initialize-world)

and make the robot run around with

In [ ]:
(discover-world)

or test the functions separately with 

In [ ]:
(defun test-place-object ()
    "Tests if objects can only be put on ground."
    (initialize-walls)
    (place-object 0 0 'test)
    (place-object 2 2 'test)
    (unless (eq (aref *world-map* 0 0) 'w)
        (format T "test-place-object: Expected 'w on tile 0 0 but got ~a" (aref *world-map* 0 0)))
    (unless (eq (aref *world-map* 2 2) 'test)
        (format T "test-place-object: Expected 'test on tile 2 2 but got ~a" (aref *world-map* 2 2))))

(test-place-object)

In [ ]:
(defun test-place-robot ()
    "Tests if the robot can only be put on ground."
    (initialize-walls)
    (place-robot 0 0)
    (place-robot 2 2)
    (unless (eq (aref *world-map* 0 0) 'w)
        (format T "test-place-robot: Expected 'w on tile 0 0 but got ~a" (aref *world-map* 0 0)))
    (unless (eq (aref *world-map* 2 2) 'r)
        (format T "test-place-robot: Expected 'r on tile 2 2 but got ~a" (aref *world-map* 2 2))))

(test-place-robot)

In [ ]:
(defun test-initialize-world ()
    "Tests if the robot and all treasures are placed in the world."
    (initialize-world)
    (let ((n (alexandria:iota (array-total-size *world-map*))))
         (dolist (treasure (cons 'r  *treasures*))
             (unless (some #'(lambda (i) (eq treasure (row-major-aref *world-map* i))) n)
                 (format T "test-initialize-world: Object ~a not found in the world." treasure)))))

(test-initialize-world)

In [ ]:
(defun test-symbol-at-pos ()
    "Tests if return value is correct."
    (initialize-walls)
    (unless (symbol-at-pos 0 0 'w)
        (format T "test-symbol-at-pos: Expected return value with coordinate 0 0 was T but got nil."))
    (when (symbol-at-pos 2 2 'fail)
        (format T "test-symbol-at-pos: Expected return value with coordinate 2 2 was nil but got T.")))

(test-symbol-at-pos)

In [ ]:
(defun test-find-object-coordinates ()
    "Tests if coordinates are found correctly."
    (initialize-walls)
    (setf (aref *world-map* 2 2) 'test)
    (unless (equal (find-object-coordinates 'test) '(2 2))
        (format T "test-find-object-coordinate: Expected (2 2) but got ~a." (find-object-coordinates 'test))))

(test-find-object-coordinates)

In [ ]:
(defun test-move ()
    "Tests if robot won't move on wall tile."
    (initialize-walls)
    (setf *battery-left* +battery-capacity+
          *treasures-found* 0
          *treasures* '('test))    
    (move-robot 0 0)
    (unless (eq *battery-left* +battery-capacity+)
        (format T "test-move-wall: Expected *battery-left* to be ~a but got ~a." +battery-capacity+ *battery-left*))
    (unless (eq *treasures-found* 0)
         (format T "test-move-wall: Expected *treasures-found* to be 0 but got ~a." *treasures-found*))
    (unless (eq (aref *world-map* 0 0) 'w)
         (format T "test-move-wall: Expected tile at coordinate 0 0 to be 'w but was ~a." (aref *world-map* 0 0)))
        
    "Tests if robot collects treasure"
    (initialize-walls)
    (setf *battery-left* +battery-capacity+
          *treasures-found* 0
          *treasures* '(test)
          (aref *world-map* 2 2) 'test)
    (move-robot 2 2)
    (unless (eq *battery-left* (- +battery-capacity+ 1))
        (format T "test-move-treasure: Expected *battery-left* to be ~a but got ~a." (- +battery-capacity+ 1) *battery-left*))
    (unless (eq *treasures-found* 1)
         (format T "test-move-treasure: Expected *treasures-found* to be 1 but got ~a." *treasures-found*))
    (unless (eq (aref *world-map* 2 2) 'r)
         (format T "test-move-treasure: Expected tile at coordinate 2 2 to be 'r but was ~a." (aref *world-map* 2 2)))
    

    "Tests if robot won't move if battery is empty."
    (initialize-walls)
    (setf *battery-left* 0
          *treasures-found* 0
          *treasures* '(test)
          (aref *world-map* 2 2) 'test)
    (move-robot 2 2)
    (unless (eq *battery-left* 0)
        (format T "test-move-battery: Expected *battery-left* to be 0 but got ~a." *battery-left*))
    (unless (eq *treasures-found* 0)
         (format T "test-move-battery: Expected *treasures-found* to be 0 but got ~a." *treasures-found*))
    (unless (eq (aref *world-map* 2 2) 'test)
         (format T "test-move-battery: Expected tile at coordinate 0 0 to be 'test but was ~a." (aref *world-map* 2 2))))
    
(test-move)

In [ ]:
(defun test-discover-world ()
    "Tests if the robot collects all treasures"
    (initialize-walls)
    (setf *battery-left* +battery-capacity+
          *treasures-found* 0
          *treasures* '(test test2)
          (aref *world-map* 2 2) 'test
          (aref *world-map* 3 3) 'test2)
    (discover-world)
    (unless (eq *battery-left* (- +battery-capacity+ 2))
        (format T "test-discover-world: Expected *battery-left* to be ~a but got ~a." (- +battery-capacity+ 2) *battery-left*))
    (unless (eq *treasures-found* 2)
        (format T "test-discover-world: Expected *treasures-found* to be 2 but got ~a." *treasures-found*)))

(test-discover-world)